# Download libraries

In [1]:
!pip install pandas
!pip install numpy
!pip install gingerit
!pip install Unidecode
!pip install word2number
!pip install contractions
!pip install beautifulsoup4
!python -m spacy download en_core_web_sm

  Using cached https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.1.0/en_core_web_sm-3.1.0-py3-none-any.whl (13.6 MB)
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


# Import libraries

In [1]:
import json
import pandas as pd
import re
import string
import random

In [2]:
import spacy
import en_core_web_sm
import numpy as np
import math
from gingerit.gingerit import GingerIt

In [3]:
import unidecode
from word2number import w2n
import contractions
from bs4 import BeautifulSoup

In [222]:
# create directory for result
from pathlib import Path
Path("../result/").mkdir(parents=True, exist_ok=True)

# Utility function

In [134]:
# utility function

# split into sentences, list of all reviews
def split_into_sentence(all_review, path):
    # use spacy to split sentences
    nlp = spacy.load('en_core_web_sm')
    
    all_sentences = []
    count = 0
    num_sentences = 0
    
    for t in all_review:
        if count%10 == 0:
            print(count)
        count += 1

        doc = nlp(t)
        sentences = list(doc.sents)
        sentences = [i.text for i in sentences]
        all_sentences.extend(sentences)

    my_df = pd.DataFrame(all_sentences)
    my_df.to_csv(path, index=False, header=False)
    return my_df


# consituency parsing into csv
def split_into_sentence_parse_sf_csv(all_review, path):
    # use spacy to split sentences
    nlp = spacy.load('en_core_web_sm')
    
    parse_dict = {}
    all_sentences = []
    count = 0
    num_sentences = 0
    
    for t in all_review:
        if count%10 == 0:
            print(count)
        count += 1

        doc = nlp(t)
        sentences = list(doc.sents)
        sentences = [i.text for i in sentences]
        
        for s in sentences:
            num_sentences += 1
            adjp_list = parse_using_standford(tuple([s, ""]))
            parse_dict[s] = ', '.join(adjp_list)

    print('num sentences {}'.format(num_sentences))
    my_df = pd.DataFrame([parse_dict]).transpose()
    my_df.to_csv(path)

# Preprocessing functions

In [27]:
def remove_accented_chars(text):
    """remove accented characters from text, e.g. café"""
    text = unidecode.unidecode(text)
    return text

def expand_contractions(text):
    """expand shortened words, e.g. don't to do not"""
    text = contractions.fix(text)
    return text

def strip_html_tags(text):
    """remove html tags from text"""
    soup = BeautifulSoup(text, "html.parser")
    stripped_text = soup.get_text(separator=" ")
    return stripped_text

def remove_non_word_char(text):
    text = re.sub('[\W]+', '', text.lower())
    return text

def remove_extra_spaces(text):
    text = re.sub(r' +', ' ', text)
    return text

def remove_emo(text):
    emoticons = find_emo(text)
    for e in emoticons:
        text = text.replace(e, '')
    return text

def find_emo(text):
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)',text)
    return emoticons

# Extraction functions

In [42]:
from  nltk.parse.corenlp import CoreNLPParser
parser = CoreNLPParser(url='http://localhost:9000')

grammar_spell_checker = GingerIt()

# methods to do consistuency parsing with stanford core
def connect():
    parser = CoreNLPParser(url='http://localhost:9000')
    print('reconnecting to standford core server ... ')
    return parser

# preprocessing done before splitting into respective sentences
def preprocessing(text):
    # noise reduction
    
    # remove html text
    text = strip_html_tags(text)
    
    # remove emoji
    text = remove_emo(text)
    
    text = remove_accented_chars(text)
    
    text = expand_contractions(text)

    # remove white spaces
    text = remove_extra_spaces(text)
    
    # lower case
    text = text.lower()
    
    return text
    

def check_grammar_spelling(s):
    new_sentence_list = []
    try:
        r = grammar_spell_checker.parse(s)
    except KeyError:
        pass
        print('pass exception')
        return s
    return r['result']

def parse_using_standford(sentence_tuple, shallow=False):
    result_list = []
    sentence_tuple = tuple([preprocessing(i) for i in sentence_tuple])
    sentences = parser.raw_parse_sents(sentence_tuple)
    for line in sentences:
        for sentence in line:
            # change this shallow
            if shallow:
                have_adjp, adjp_list = traverse_tree_sf_shallow(sentence)
            else:
                have_adjp, adjp_list = traverse_tree_sf(sentence)
            result_list.extend(adjp_list)
    return result_list

def traverse_tree_sf_shallow(t):
    try:
        t.label()
    except AttributeError:
        return
    adjp_list = []
    subtree_have_adjp = False
    if t.label().find("ADJP") != -1:
        return True, [" ".join(t.leaves())]
    for subtree in t:
        have_adjp = False
        if subtree.height() != 2:
            have_adjp, adjp_list_subtree = traverse_tree_sf_shallow(subtree)
            
        if (have_adjp):
            subtree_have_adjp = True
            adjp_list.extend(adjp_list_subtree)

    return subtree_have_adjp, adjp_list

def traverse_tree_sf(t):
    try:
        t.label()
    except AttributeError:
        return
    adjp_list = []
    subtree_have_adjp = False
    
    for subtree in t:
        have_adjp = False
        if subtree.height() != 2:
            have_adjp, adjp_list_subtree = traverse_tree_sf(subtree)
            
        if (have_adjp):
            subtree_have_adjp = True
            adjp_list.extend(adjp_list_subtree)

    if t.label().find("ADJP") != -1:
        if (subtree_have_adjp):
            return True, adjp_list
        else:
            return True, [" ".join(t.leaves())]
    else:
        return subtree_have_adjp, adjp_list

# process df
def process_text_split_sentence_sf(df, spelling_grammar_checker=False, shallow=False):
    # use spacy to split sentences
    nlp = spacy.load('en_core_web_sm')
    text = df['text'].astype('str')
    count = 0
    num_sentences = 0
    print('Number of text {}'.format(len(text)))
    adjp_dict={}
    batch = 500
    for i in range(batch, len(text)+1, batch):
        print('batch {} from {}'.format(i/batch, i))
        for r in range(batch):
            print('text id {}'.format(i-batch+r))
            if spelling_grammar_checker:
                cur = check_grammar_spelling(text[i-batch+r])
            else:
                cur = text[i-batch+r]
            doc = nlp(cur)
            generator = list(doc.sents)
            sentence_tuple = [x.text for x in doc.sents]
            sentence_tuple = tuple(sentence_tuple)
            adjp_list = parse_using_standford(sentence_tuple, shallow)
            for x in range(len(adjp_list)):
                key = adjp_list[x].lower()  # change everything to lower
                if key in adjp_dict:
                    adjp_dict[key] += 1
                else:
                    adjp_dict[key] = 1
        
        # save into json
        with open('../result/run9/adjp_sf_batch_{}.json'.format(i), 'w') as file:
            file.write(json.dumps(adjp_dict)) # use `json.loads` to do the reverse
            
        # connect to server again
        parser = connect()
    return adjp_dict

# processs list of strings
def process_plain_text(all_review, shallow=False):
    parser = connect()
    # use spacy to split sentences
    nlp = spacy.load('en_core_web_sm')

    count = 0
    num_sentences = 0
    print('Number of text {}'.format(len(all_review)))
    adjp_dict={}
    for i in range(len(all_review)):
        print('text id {}'.format(i))
        doc = nlp(all_review[i])
        generator = list(doc.sents)
        sentence_tuple = [x.text for x in doc.sents]
        sentence_tuple = tuple(sentence_tuple)
        
        adjp_list = parse_using_standford(sentence_tuple, shallow)
        for x in range(len(adjp_list)):
            key = adjp_list[x].lower()  # change everything to lower
            if key in adjp_dict:
                adjp_dict[key] += 1
            else:
                adjp_dict[key] = 1
        
    return adjp_dict

# Functions for postprocessing

In [158]:
# post processing of data
def remove_all_punctuation(data_dict):
    new_dict = {}
    count = 0
    for k, v in data_dict.items():
        new_key = k.translate(str.maketrans('', '', string.punctuation))
        if new_key in new_dict:
            count += 1
            new_dict[new_key] += 1
        else:
            new_dict[new_key] = v
    print('Number of changes: {}'.format(count))
    return new_dict

def remove_adj_dict(data_dict):
    remove_list = []
    for k in data_dict:
        if len(k.split()) == 1:
            remove_list.append(k)
    
    for i in remove_list:
        del data_dict[i]
    return data_dict

def remove_extra_spaces_dict(data_dict):
    new_dict = {}
    for k,v in data_dict.items():
        new_dict[remove_extra_spaces(k).rstrip()] = v
    return new_dict

# descending
def sort_dict(data_dict):
    return {k: v for k, v in sorted(data_dict.items(), key=lambda item: item[1], reverse=True)}

def remove_adj_list(data_list):
    count = 0
    new_data_list = []
    for k in data_list:
        if len(k.split()) > 1:
            new_data_list.append(k)
        else:
            count += 1
    print('Number of adj removed: {}'.format(count))
    return new_data_list

def remove_extra_spaces_list(data_list):
    new_list = []
    for k in data_list:
        new_list.append(remove_extra_spaces(k).rstrip())
    return new_list

# post processing of data
def remove_all_punctuation_list(data_list):
    new_data_list = []
    count = 0
    for k in data_list:
        new_key = k.translate(str.maketrans('', '', string.punctuation))
        if new_key not in new_data_list:
            new_data_list.append(new_key)
            count += 1
    print('Number of changes: {}'.format(count))
    return new_data_list

def postprocessing_dict(data_dict):
    print('dict size before processing: {}'.format(len(data_dict)))
    data_dict = remove_all_punctuation(data_dict)
    data_dict = remove_extra_spaces_dict(data_dict)
    data_dict = remove_adj_dict(data_dict)
    print('dict size aftter processing: {}'.format(len(data_dict)))
    return sort_dict(data_dict)

def postprocessing_list(data_list):
    print('list size before processing: {}'.format(len(data_list)))
    data_list = remove_all_punctuation_list(data_list)
    data_list = remove_extra_spaces_list(data_list)
    data_list = remove_adj_list(data_list)
    print('list size aftter processing: {}'.format(len(data_list)))
    return data_list

# Function for getting biz & get indicative adjp

In [8]:
def get_random_business(df):
    unique_biz = df['business_id'].unique()
    print('unique biz {}'.format(len(unique_biz)))
    random_biz = random.choice(unique_biz)
    print('random biz choosen: {}'.format(random_biz))
    return random_biz

def get_all_reviews_from_biz(biz):
    rows = df.loc[df['business_id'] == biz]
    text = rows['text'].astype('str')
    text_list = rows['text'].tolist()
    print('Number of reviews: {}'.format(len(rows)))
    return tuple(text_list)

def get_indicative_adjp(common_adjp, biz_adjp_list):
    indicative_adjp = []
    for adjp in biz_adjp_list:
        if adjp not in common_adjp:
            indicative_adjp.append(adjp)
    return indicative_adjp

In [117]:
# frequent 5%
def get_count_for_top_n_percent(data_dict, percent):
    # try stats
    data_df = pd.DataFrame(data_dict, index=['count'])
    data_df = data_df.transpose()
    stats = data_df.describe()
    
    sorted_dict = sort_dict(data_dict)
    num_5_percent = round(int(stats.iloc[0])*(percent/100))
    print(percent/100)
    print('Number of adjp that are top 5% {}'.format(num_5_percent))
    count_for_5_percent = data_dict[list(sorted_dict)[num_5_percent]]
    print('Count for top 5% {}'.format(count_for_5_percent))
    return count_for_5_percent

# diff between 2 list
def Diff(li1, li2):
    return list(set(li1) - set(li2)), list(set(li2) - set(li1))

In [10]:
def list_to_csv(a_list, path):
    my_df = pd.DataFrame(a_list)
    my_df.to_csv(path, index=False, header=False)

# Load data

In [11]:
df = pd.read_json('../data/reviewSelected100.json', lines=True, encoding = "ISO-8859-1")
text = df['text'].astype('str')

# Run stanford parsing

## Deep extraction

In [43]:
%%time
result_dict_deep = process_text_split_sentence_sf(df, False)

Number of text 15300
batch 1.0 from 500
text id 0
text id 1
text id 2
text id 3
text id 4
text id 5
text id 6
text id 7
text id 8
text id 9
text id 10
text id 11
text id 12
text id 13
text id 14
text id 15
text id 16
text id 17
text id 18
text id 19
text id 20
text id 21
text id 22
text id 23
text id 24
text id 25
text id 26
text id 27
text id 28
text id 29
text id 30
text id 31
text id 32
text id 33
text id 34
text id 35
text id 36
text id 37
text id 38
text id 39
text id 40
text id 41
text id 42
text id 43
text id 44
text id 45
text id 46
text id 47
text id 48
text id 49
text id 50
text id 51
text id 52
text id 53
text id 54
text id 55
text id 56
text id 57
text id 58
text id 59
text id 60
text id 61
text id 62
text id 63


/Users/cecilialee/miniconda3/lib/python3.9/site-packages/bs4/__init__.py:337: MarkupResemblesLocatorWarning: ".." looks like a directory name, not markup. You may want to open a file found in this directory and pass the filehandle into Beautiful Soup.
  warnings.warn(


text id 64
text id 65
text id 66
text id 67
text id 68
text id 69
text id 70
text id 71
text id 72
text id 73
text id 74
text id 75
text id 76
text id 77
text id 78
text id 79
text id 80
text id 81
text id 82
text id 83
text id 84
text id 85
text id 86
text id 87
text id 88
text id 89
text id 90
text id 91
text id 92
text id 93
text id 94
text id 95
text id 96
text id 97
text id 98
text id 99
text id 100
text id 101
text id 102
text id 103
text id 104
text id 105
text id 106
text id 107
text id 108
text id 109
text id 110
text id 111
text id 112
text id 113
text id 114
text id 115
text id 116
text id 117
text id 118
text id 119
text id 120
text id 121
text id 122
text id 123
text id 124
text id 125
text id 126
text id 127
text id 128
text id 129
text id 130
text id 131
text id 132
text id 133
text id 134
text id 135
text id 136
text id 137
text id 138
text id 139
text id 140
text id 141
text id 142
text id 143
text id 144
text id 145
text id 146
text id 147
text id 148
text id 149
text

/Users/cecilialee/miniconda3/lib/python3.9/site-packages/bs4/__init__.py:431: MarkupResemblesLocatorWarning: "http://www.fda.gov/MedicalDevices/ProductsandMedicalProcedures/SurgeryandLifeSupport/LASIK/ucm061354.htm

http://www.doctoroz.com/article/hard-look-lasik-surgery" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  warnings.warn(


text id 521
text id 522
text id 523
text id 524
text id 525
text id 526
text id 527
text id 528
text id 529
text id 530
text id 531
text id 532
text id 533
text id 534
text id 535
text id 536
text id 537
text id 538
text id 539
text id 540
text id 541
text id 542
text id 543
text id 544
text id 545
text id 546
text id 547
text id 548
text id 549
text id 550
text id 551
text id 552
text id 553
text id 554
text id 555
text id 556
text id 557
text id 558
text id 559
text id 560
text id 561
text id 562
text id 563
text id 564
text id 565
text id 566
text id 567
text id 568
text id 569
text id 570
text id 571
text id 572
text id 573
text id 574
text id 575
text id 576
text id 577
text id 578
text id 579
text id 580
text id 581
text id 582
text id 583
text id 584
text id 585
text id 586
text id 587
text id 588
text id 589
text id 590
text id 591
text id 592
text id 593
text id 594
text id 595
text id 596
text id 597
text id 598
text id 599
text id 600
text id 601
text id 602
text id 603
text

...

/Users/cecilialee/miniconda3/lib/python3.9/site-packages/bs4/__init__.py:337: MarkupResemblesLocatorWarning: "/" looks like a directory name, not markup. You may want to open a file found in this directory and pass the filehandle into Beautiful Soup.
  warnings.warn(


text id 12751
text id 12752
text id 12753
text id 12754
text id 12755
text id 12756
text id 12757
text id 12758
text id 12759
text id 12760
text id 12761
text id 12762
text id 12763
text id 12764
text id 12765
text id 12766
text id 12767
text id 12768
text id 12769
text id 12770
text id 12771
text id 12772
text id 12773
text id 12774
text id 12775
text id 12776
text id 12777
text id 12778
text id 12779
text id 12780
text id 12781
text id 12782
text id 12783
text id 12784
text id 12785
text id 12786
text id 12787
text id 12788
text id 12789
text id 12790
text id 12791
text id 12792
text id 12793
text id 12794
text id 12795
text id 12796
text id 12797
text id 12798
text id 12799
text id 12800
text id 12801
text id 12802
text id 12803
text id 12804
text id 12805
text id 12806
text id 12807
text id 12808
text id 12809
text id 12810
text id 12811
text id 12812
text id 12813
text id 12814
text id 12815
text id 12816
text id 12817
text id 12818
text id 12819
text id 12820
text id 12821
text i

text id 13332
text id 13333
text id 13334
text id 13335
text id 13336
text id 13337
text id 13338
text id 13339
text id 13340
text id 13341
text id 13342
text id 13343
text id 13344
text id 13345
text id 13346
text id 13347
text id 13348
text id 13349
text id 13350
text id 13351
text id 13352
text id 13353
text id 13354
text id 13355
text id 13356
text id 13357
text id 13358
text id 13359
text id 13360
text id 13361
text id 13362
text id 13363
text id 13364
text id 13365
text id 13366
text id 13367
text id 13368
text id 13369
text id 13370
text id 13371
text id 13372
text id 13373
text id 13374
text id 13375
text id 13376
text id 13377
text id 13378
text id 13379
text id 13380
text id 13381
text id 13382
text id 13383
text id 13384
text id 13385
text id 13386
text id 13387
text id 13388
text id 13389
text id 13390
text id 13391
text id 13392
text id 13393
text id 13394
text id 13395
text id 13396
text id 13397
text id 13398
text id 13399
text id 13400
text id 13401
text id 13402
text i

text id 13913
text id 13914
text id 13915
text id 13916
text id 13917
text id 13918
text id 13919
text id 13920
text id 13921
text id 13922
text id 13923
text id 13924
text id 13925
text id 13926
text id 13927
text id 13928
text id 13929
text id 13930
text id 13931
text id 13932
text id 13933
text id 13934
text id 13935
text id 13936
text id 13937
text id 13938
text id 13939
text id 13940
text id 13941
text id 13942
text id 13943
text id 13944
text id 13945
text id 13946
text id 13947
text id 13948
text id 13949
text id 13950
text id 13951
text id 13952
text id 13953
text id 13954
text id 13955
text id 13956
text id 13957
text id 13958
text id 13959
text id 13960
text id 13961
text id 13962
text id 13963
text id 13964
text id 13965
text id 13966
text id 13967
text id 13968
text id 13969
text id 13970
text id 13971
text id 13972
text id 13973
text id 13974
text id 13975
text id 13976
text id 13977
text id 13978
text id 13979
text id 13980
text id 13981
text id 13982
text id 13983
text i

text id 14495
text id 14496
text id 14497
text id 14498
text id 14499
reconnecting to standford core server ... 
batch 30.0 from 15000
text id 14500
text id 14501
text id 14502
text id 14503
text id 14504
text id 14505
text id 14506
text id 14507
text id 14508
text id 14509
text id 14510
text id 14511
text id 14512
text id 14513
text id 14514
text id 14515
text id 14516
text id 14517
text id 14518
text id 14519
text id 14520
text id 14521
text id 14522
text id 14523
text id 14524
text id 14525
text id 14526
text id 14527
text id 14528
text id 14529
text id 14530
text id 14531
text id 14532
text id 14533
text id 14534
text id 14535
text id 14536
text id 14537
text id 14538
text id 14539
text id 14540
text id 14541
text id 14542
text id 14543
text id 14544
text id 14545
text id 14546
text id 14547
text id 14548
text id 14549
text id 14550
text id 14551
text id 14552
text id 14553
text id 14554
text id 14555
text id 14556
text id 14557
text id 14558
text id 14559
text id 14560
text id 145

## Shallow extraction

In [45]:
%%time
result_dict_shallow = process_text_split_sentence_sf(df, False, True)

Number of text 15300
batch 1.0 from 500
text id 0
text id 1
text id 2
text id 3
text id 4
text id 5
text id 6
text id 7
text id 8
text id 9
text id 10
text id 11
text id 12
text id 13
text id 14
text id 15
text id 16
text id 17
text id 18
text id 19
text id 20
text id 21
text id 22
text id 23
text id 24
text id 25
text id 26
text id 27
text id 28
text id 29
text id 30
text id 31
text id 32
text id 33
text id 34
text id 35
text id 36
text id 37
text id 38
text id 39
text id 40
text id 41
text id 42
text id 43
text id 44
text id 45
text id 46
text id 47
text id 48
text id 49
text id 50
text id 51
text id 52
text id 53
text id 54
text id 55
text id 56
text id 57
text id 58
text id 59
text id 60
text id 61
text id 62
text id 63
text id 64
text id 65
text id 66
text id 67
text id 68
text id 69
text id 70
text id 71
text id 72
text id 73
text id 74
text id 75
text id 76
text id 77
text id 78
text id 79
text id 80
text id 81
text id 82
text id 83
text id 84
text id 85
text id 86
text id 87
te

text id 684
text id 685
text id 686
text id 687
text id 688
text id 689
text id 690
text id 691
text id 692
text id 693
text id 694
text id 695
text id 696
text id 697
text id 698
text id 699
text id 700
text id 701
text id 702
text id 703
text id 704
text id 705
text id 706
text id 707
text id 708
text id 709
text id 710
text id 711
text id 712
text id 713
text id 714
text id 715
text id 716
text id 717
text id 718
text id 719
text id 720
text id 721
text id 722
text id 723
text id 724
text id 725
text id 726
text id 727
text id 728
text id 729
text id 730
text id 731
text id 732
text id 733
text id 734
text id 735
text id 736
text id 737
text id 738
text id 739
text id 740
text id 741
text id 742
text id 743
text id 744
text id 745
text id 746
text id 747
text id 748
text id 749
text id 750
text id 751
text id 752
text id 753
text id 754
text id 755
text id 756
text id 757
text id 758
text id 759
text id 760
text id 761
text id 762
text id 763
text id 764
text id 765
text id 766
text

text id 1334
text id 1335
text id 1336
text id 1337
text id 1338
text id 1339
text id 1340
text id 1341
text id 1342
text id 1343
text id 1344
text id 1345
text id 1346
text id 1347
text id 1348
text id 1349
text id 1350
text id 1351
text id 1352
text id 1353
text id 1354
text id 1355
text id 1356
text id 1357
text id 1358
text id 1359
text id 1360
text id 1361
text id 1362
text id 1363
text id 1364
text id 1365
text id 1366
text id 1367
text id 1368
text id 1369
text id 1370
text id 1371
text id 1372
text id 1373
text id 1374
text id 1375
text id 1376
text id 1377
text id 1378
text id 1379
text id 1380
text id 1381
text id 1382
text id 1383
text id 1384
text id 1385
text id 1386
text id 1387
text id 1388
text id 1389
text id 1390
text id 1391
text id 1392
text id 1393
text id 1394
text id 1395
text id 1396
text id 1397
text id 1398
text id 1399
text id 1400
text id 1401
text id 1402
text id 1403
text id 1404
text id 1405
text id 1406
text id 1407
text id 1408
text id 1409
text id 1410

text id 1962
text id 1963
text id 1964
text id 1965
text id 1966
text id 1967
text id 1968
text id 1969
text id 1970
text id 1971
text id 1972
text id 1973
text id 1974
text id 1975
text id 1976
text id 1977
text id 1978
text id 1979
text id 1980
text id 1981
text id 1982
text id 1983
text id 1984
text id 1985
text id 1986
text id 1987
text id 1988
text id 1989
text id 1990
text id 1991
text id 1992
text id 1993
text id 1994
text id 1995
text id 1996
text id 1997
text id 1998
text id 1999
reconnecting to standford core server ... 
batch 5.0 from 2500
text id 2000
text id 2001
text id 2002
text id 2003
text id 2004
text id 2005
text id 2006
text id 2007
text id 2008
text id 2009
text id 2010
text id 2011
text id 2012
text id 2013
text id 2014
text id 2015
text id 2016
text id 2017
text id 2018
text id 2019
text id 2020
text id 2021
text id 2022
text id 2023
text id 2024
text id 2025
text id 2026
text id 2027
text id 2028
text id 2029
text id 2030
text id 2031
text id 2032
text id 2033
t

text id 2585
text id 2586
text id 2587
text id 2588
text id 2589
text id 2590
text id 2591
text id 2592
text id 2593
text id 2594
text id 2595
text id 2596
text id 2597
text id 2598
text id 2599
text id 2600
text id 2601
text id 2602
text id 2603
text id 2604
text id 2605
text id 2606
text id 2607
text id 2608
text id 2609
text id 2610
text id 2611
text id 2612
text id 2613
text id 2614
text id 2615
text id 2616
text id 2617
text id 2618
text id 2619
text id 2620
text id 2621
text id 2622
text id 2623
text id 2624
text id 2625
text id 2626
text id 2627
text id 2628
text id 2629
text id 2630
text id 2631
text id 2632
text id 2633
text id 2634
text id 2635
text id 2636
text id 2637
text id 2638
text id 2639
text id 2640
text id 2641
text id 2642
text id 2643
text id 2644
text id 2645
text id 2646
text id 2647
text id 2648
text id 2649
text id 2650
text id 2651
text id 2652
text id 2653
text id 2654
text id 2655
text id 2656
text id 2657
text id 2658
text id 2659
text id 2660
text id 2661

...

text id 12409
text id 12410
text id 12411
text id 12412
text id 12413
text id 12414
text id 12415
text id 12416
text id 12417
text id 12418
text id 12419
text id 12420
text id 12421
text id 12422
text id 12423
text id 12424
text id 12425
text id 12426
text id 12427
text id 12428
text id 12429
text id 12430
text id 12431
text id 12432
text id 12433
text id 12434
text id 12435
text id 12436
text id 12437
text id 12438
text id 12439
text id 12440
text id 12441
text id 12442
text id 12443
text id 12444
text id 12445
text id 12446
text id 12447
text id 12448
text id 12449
text id 12450
text id 12451
text id 12452
text id 12453
text id 12454
text id 12455
text id 12456
text id 12457
text id 12458
text id 12459
text id 12460
text id 12461
text id 12462
text id 12463
text id 12464
text id 12465
text id 12466
text id 12467
text id 12468
text id 12469
text id 12470
text id 12471
text id 12472
text id 12473
text id 12474
text id 12475
text id 12476
text id 12477
text id 12478
text id 12479
text i

text id 12991
text id 12992
text id 12993
text id 12994
text id 12995
text id 12996
text id 12997
text id 12998
text id 12999
reconnecting to standford core server ... 
batch 27.0 from 13500
text id 13000
text id 13001
text id 13002
text id 13003
text id 13004
text id 13005
text id 13006
text id 13007
text id 13008
text id 13009
text id 13010
text id 13011
text id 13012
text id 13013
text id 13014
text id 13015
text id 13016
text id 13017
text id 13018
text id 13019
text id 13020
text id 13021
text id 13022
text id 13023
text id 13024
text id 13025
text id 13026
text id 13027
text id 13028
text id 13029
text id 13030
text id 13031
text id 13032
text id 13033
text id 13034
text id 13035
text id 13036
text id 13037
text id 13038
text id 13039
text id 13040
text id 13041
text id 13042
text id 13043
text id 13044
text id 13045
text id 13046
text id 13047
text id 13048
text id 13049
text id 13050
text id 13051
text id 13052
text id 13053
text id 13054
text id 13055
text id 13056
text id 130

text id 13567
text id 13568
text id 13569
text id 13570
text id 13571
text id 13572
text id 13573
text id 13574
text id 13575
text id 13576
text id 13577
text id 13578
text id 13579
text id 13580
text id 13581
text id 13582
text id 13583
text id 13584
text id 13585
text id 13586
text id 13587
text id 13588
text id 13589
text id 13590
text id 13591
text id 13592
text id 13593
text id 13594
text id 13595
text id 13596
text id 13597
text id 13598
text id 13599
text id 13600
text id 13601
text id 13602
text id 13603
text id 13604
text id 13605
text id 13606
text id 13607
text id 13608
text id 13609
text id 13610
text id 13611
text id 13612
text id 13613
text id 13614
text id 13615
text id 13616
text id 13617
text id 13618
text id 13619
text id 13620
text id 13621
text id 13622
text id 13623
text id 13624
text id 13625
text id 13626
text id 13627
text id 13628
text id 13629
text id 13630
text id 13631
text id 13632
text id 13633
text id 13634
text id 13635
text id 13636
text id 13637
text i

text id 14148
text id 14149
text id 14150
text id 14151
text id 14152
text id 14153
text id 14154
text id 14155
text id 14156
text id 14157
text id 14158
text id 14159
text id 14160
text id 14161
text id 14162
text id 14163
text id 14164
text id 14165
text id 14166
text id 14167
text id 14168
text id 14169
text id 14170
text id 14171
text id 14172
text id 14173
text id 14174
text id 14175
text id 14176
text id 14177
text id 14178
text id 14179
text id 14180
text id 14181
text id 14182
text id 14183
text id 14184
text id 14185
text id 14186
text id 14187
text id 14188
text id 14189
text id 14190
text id 14191
text id 14192
text id 14193
text id 14194
text id 14195
text id 14196
text id 14197
text id 14198
text id 14199
text id 14200
text id 14201
text id 14202
text id 14203
text id 14204
text id 14205
text id 14206
text id 14207
text id 14208
text id 14209
text id 14210
text id 14211
text id 14212
text id 14213
text id 14214
text id 14215
text id 14216
text id 14217
text id 14218
text i

text id 14729
text id 14730
text id 14731
text id 14732
text id 14733
text id 14734
text id 14735
text id 14736
text id 14737
text id 14738
text id 14739
text id 14740
text id 14741
text id 14742
text id 14743
text id 14744
text id 14745
text id 14746
text id 14747
text id 14748
text id 14749
text id 14750
text id 14751
text id 14752
text id 14753
text id 14754
text id 14755
text id 14756
text id 14757
text id 14758
text id 14759
text id 14760
text id 14761
text id 14762
text id 14763
text id 14764
text id 14765
text id 14766
text id 14767
text id 14768
text id 14769
text id 14770
text id 14771
text id 14772
text id 14773
text id 14774
text id 14775
text id 14776
text id 14777
text id 14778
text id 14779
text id 14780
text id 14781
text id 14782
text id 14783
text id 14784
text id 14785
text id 14786
text id 14787
text id 14788
text id 14789
text id 14790
text id 14791
text id 14792
text id 14793
text id 14794
text id 14795
text id 14796
text id 14797
text id 14798
text id 14799
text i

# Post processing of adjp from deep extraction

In [159]:
result_dict_deep_p = postprocessing_dict(result_dict_deep)

dict size before processing: 26827
Number of changes: 86
dict size aftter processing: 24191


In [210]:
sort_dict(result_dict_deep_p)

{'much better': 166,
 'so much': 156,
 'so many': 148,
 'not sure': 93,
 'how much': 90,
 'as good': 70,
 'very busy': 69,
 'well worth': 61,
 'super nice': 59,
 'how many': 57,
 'quite good': 56,
 'good too': 56,
 'pretty decent': 55,
 'really great': 54,
 'worth it': 53,
 'very little': 51,
 'very small': 51,
 'highly recommended': 50,
 'even better': 49,
 'too busy': 47,
 'friendly and helpful': 44,
 'great too': 41,
 'how good': 41,
 'too many': 41,
 'just okay': 41,
 'needless to say': 38,
 'really cool': 37,
 'very sweet': 36,
 'very large': 35,
 'very accommodating': 34,
 'very comfortable': 34,
 'way better': 33,
 'too long': 32,
 'so much better': 32,
 'so sweet': 32,
 'a little more': 32,
 'fresh and delicious': 32,
 'really bad': 31,
 'so fresh': 31,
 'consistently good': 31,
 'really friendly': 30,
 'not worth': 30,
 'hot and sour': 30,
 'really busy': 29,
 'most recent': 29,
 'as much': 28,
 'super busy': 28,
 'pretty tasty': 28,
 'fast and friendly': 27,
 'extremely frien

In [215]:
result_deep_df = pd.DataFrame(result_dict_deep_p, index=['frequency']).transpose()
result_deep_df.to_csv('../result/frequency_result_deep_extraction.csv')

# Get frequently-used ADJP deep extraction

In [160]:
frequently_used_adjp_deep = get_frequently_used_adjp(result_dict_deep_p)

0.05
Number of adjp that are top 5% 1210
Count for top 5% 3
Num adjp considered common: 1493


In [221]:
frequently_used_adjp_deep_df = pd.DataFrame(frequently_used_adjp_deep)
frequently_used_adjp_deep_df.to_csv('../result/frequently_used_adjp_deep_extraction.csv')

# post processing of adjp from shallow extraction

In [161]:
result_dict_shallow_p = postprocessing_dict(result_dict_shallow)

dict size before processing: 29437
Number of changes: 86
dict size aftter processing: 27047


In [162]:
len(result_dict_shallow_p)

27047

In [211]:
sort_dict(result_dict_shallow_p)

{'so much': 148,
 'so many': 133,
 'much better': 107,
 'not sure': 81,
 'how much': 78,
 'very busy': 62,
 'worth it': 59,
 'as good': 57,
 'quite good': 53,
 'good too': 53,
 'really great': 53,
 'how many': 52,
 'pretty decent': 50,
 'well worth': 50,
 'highly recommended': 49,
 'very little': 44,
 'too busy': 44,
 'friendly and helpful': 44,
 'very small': 43,
 'super nice': 43,
 'great too': 38,
 'needless to say': 38,
 'too many': 38,
 'just okay': 38,
 'how good': 37,
 'really cool': 36,
 'even better': 33,
 'very sweet': 32,
 'fresh and delicious': 32,
 'very large': 30,
 'hot and sour': 30,
 'really bad': 29,
 'too long': 29,
 'very comfortable': 29,
 'really friendly': 28,
 'a little more': 28,
 'medium rare': 27,
 'really busy': 27,
 'consistently good': 27,
 'very accommodating': 27,
 'pretty tasty': 27,
 'pretty cool': 27,
 'fast and friendly': 26,
 'as much': 26,
 'most recent': 26,
 'so glad': 26,
 'so sweet': 26,
 'super busy': 25,
 'pretty bad': 24,
 'so much better': 

In [214]:
result_shallow_df = pd.DataFrame(result_dict_shallow_p, index=['frequency']).transpose()
result_shallow_df.to_csv('../result/frequency_result_shallow_extraction.csv')

# Get frequently-used ADJP - shallow extraction

In [163]:
def get_frequently_used_adjp(data_dict):
    frequent_count = get_count_for_top_n_percent(data_dict, 5)  # top 5 %
    common_adjp = [k for k, v in data_dict.items() if v >= frequent_count]
    print('Num adjp considered common: {}'.format(len(common_adjp)))
    return common_adjp

In [164]:
frequently_used_adjp_shallow = get_frequently_used_adjp(result_dict_shallow_p)

0.05
Number of adjp that are top 5% 1352
Count for top 5% 2
Num adjp considered common: 2652


In [219]:
frequently_used_adjp_shallow_df = pd.DataFrame(frequently_used_adjp_shallow)
frequently_used_adjp_shallow_df.to_csv('../result/frequently_used_adjp_shallow_extraction.csv')

# Get random biz & extract adjp from their reviews

In [46]:
# random_biz = get_random_business(df)

random_biz = 'VWb8gk_DKUCDKw3Xsdq8Jg'
all_reviews = get_all_reviews_from_biz(random_biz)
adjp_from_biz_shallow_parsing = process_plain_text(all_reviews, True)

Number of reviews: 100
reconnecting to standford core server ... 
Number of text 100
text id 0
text id 1
text id 2
text id 3
text id 4
text id 5
text id 6
text id 7
text id 8
text id 9
text id 10
text id 11
text id 12
text id 13
text id 14
text id 15
text id 16
text id 17
text id 18
text id 19
text id 20
text id 21
text id 22
text id 23
text id 24
text id 25
text id 26
text id 27
text id 28
text id 29
text id 30
text id 31
text id 32
text id 33
text id 34
text id 35
text id 36
text id 37
text id 38
text id 39
text id 40
text id 41
text id 42
text id 43
text id 44
text id 45
text id 46
text id 47
text id 48
text id 49
text id 50
text id 51
text id 52
text id 53
text id 54
text id 55
text id 56
text id 57
text id 58
text id 59
text id 60
text id 61
text id 62
text id 63
text id 64
text id 65
text id 66
text id 67
text id 68
text id 69
text id 70
text id 71
text id 72
text id 73
text id 74
text id 75
text id 76
text id 77
text id 78
text id 79
text id 80
text id 81
text id 82
text id 83
t

## Postprocessing

In [165]:
adjp_from_biz_shallow_parsing_p = postprocessing_list(adjp_from_biz_shallow_parsing)

list size before processing: 354
Number of changes: 354
Number of adj removed: 103
list size aftter processing: 251


# Compare with frequently-used adjp to get indicative adjp

### Deep extraction

In [166]:
indicative_adjp_sf_deep = get_indicative_adjp(frequently_used_adjp_deep, adjp_from_biz_shallow_parsing_p)

In [167]:
len(indicative_adjp_sf_deep)

174

In [168]:
indicative_adjp_sf_deep

['disappointed in my waffle',
 'so long for my waffle',
 'cold out of gate',
 'overly busy',
 'so excited for them to have a move convenient for me location',
 '4 year old',
 'weird with him',
 'disappointed with my experience',
 'plentiful with smiles on their faces even during a rush',
 'open at 8 am for breakfast brunch',
 'absolutely delicious with a nice kick of heat',
 'very slow',
 'light and flavorful',
 'so good',
 'slow to seat us despite there being many empty tables',
 'really one',
 'peppermint ok',
 'better than the service but not necessarily enough',
 'savory to sweet',
 'moldy and rotten',
 'very short with us and very miserable',
 'back to waffles inc',
 'just decent',
 'a bit softer than i like',
 'highly disappointed this time considering my first experience here was great',
 'pleasantly surprised',
 'nice sized',
 'so great',
 'a little small for my preference but the coffee itself is great',
 'fine in summer',
 'too much about the hospitality industry',
 'differen

### Shallow extraction

In [169]:
indicative_adjp_sf_shallow = get_indicative_adjp(frequently_used_adjp_shallow, adjp_from_biz_shallow_parsing_p)

In [170]:
indicative_adjp_sf_shallow

['disappointed in my waffle',
 'so long for my waffle',
 'cold out of gate',
 'overly busy',
 'so excited for them to have a move convenient for me location',
 '4 year old',
 'weird with him',
 'disappointed with my experience',
 'plentiful with smiles on their faces even during a rush',
 'open at 8 am for breakfast brunch',
 'absolutely delicious with a nice kick of heat',
 'very slow',
 'light and flavorful',
 'slow to seat us despite there being many empty tables',
 'really one',
 'peppermint ok',
 'better than the service but not necessarily enough',
 'savory to sweet',
 'moldy and rotten',
 'very short with us and very miserable',
 'back to waffles inc',
 'a bit softer than i like',
 'highly disappointed this time considering my first experience here was great',
 'pleasantly surprised',
 'nice sized',
 'so great',
 'a little small for my preference but the coffee itself is great',
 'fine in summer',
 'too much about the hospitality industry',
 'different than many',
 'watered down

In [171]:
len(indicative_adjp_sf_shallow)

152

## Difference between shallow and deep

In [172]:
in_shallow_not_in_deep, in_deep_not_in_shallow = Diff(indicative_adjp_sf_shallow, indicative_adjp_sf_deep)

In [173]:
in_shallow_not_in_deep

[]

In [174]:
in_deep_not_in_shallow

['pick up',
 'just decent',
 'able to order',
 'really poor',
 'so watery',
 'way too long',
 'quite satisfying',
 'a little runny in the middle',
 'how expensive',
 'thick and tasty',
 'too popular',
 'any more',
 'mediocre at best',
 'hard to decide what to get',
 'hard to decide',
 'thoroughly enjoyable',
 'friendly and informative',
 'so good',
 'patient with us',
 'slightly high',
 'very helpful and accommodating',
 'very well seasoned']

# Is adjective indicative?

Make sure that the phrases do not appear more than 2 times in 200 randomly extracted reviews - if the adjective phrase appears more than 2 times in the 200 reviews, it would be deemed as not indicative and considered as identification error as well.

In [191]:
def get_n_random_reviews_not_in_biz(df, biz, n):
    random_reviews = []
    df_without_biz = df.loc[df['business_id'] != biz]
    text = df_without_biz['text'].astype('str')
    text_list = text.tolist()
    for i in range(n):
        random_reviews.append(random.choice(text_list))
    return random_reviews

In [204]:
random_200_reviews = get_n_random_reviews_not_in_biz(df, random_biz, 200)

# join sentences in list tgt
random_200_reviews = " ".join(random_200_reviews)

random_1000_reviews = get_n_random_reviews_not_in_biz(df, random_biz, 1000)
random_1000_reviews = " ".join(random_1000_reviews)

random_2000_reviews = get_n_random_reviews_not_in_biz(df, random_biz, 2000)
random_2000_reviews = " ".join(random_2000_reviews)

### for deep extraction

In [193]:
def get_adjp_appear_more_than_n_times(random_reviews, adjp_list, n):
    adjp_appear_list = []
    for i in adjp_list:
        if len([m.start() for m in re.finditer(i, random_reviews)]) > n:
            adjp_appear_list.append(i)
    print('Number of adjp that appear more than n times: {}'.format(len(adjp_appear_list)))
    return adjp_appear_list

In [194]:
adjp_list_appear_more_deep = get_adjp_appear_more_than_n_times(random_200_reviews, indicative_adjp_sf_deep, 2)

Number of adjp that appear more than n times: 2


In [195]:
adjp_list_appear_more_deep

['so good', 'pretty good']

In [199]:
adjp_list_appear_more_deep = get_adjp_appear_more_than_n_times(random_1000_reviews, indicative_adjp_sf_deep, 2)

Number of adjp that appear more than n times: 11


In [200]:
adjp_list_appear_more_deep

['very slow',
 'so good',
 'pleasantly surprised',
 'so great',
 'so friendly',
 'just ok',
 'pretty good',
 'pick up',
 'able to order',
 'any more',
 'so hungry']

In [205]:
adjp_list_appear_more_deep = get_adjp_appear_more_than_n_times(random_2000_reviews, indicative_adjp_sf_deep, 2)

Number of adjp that appear more than n times: 13


In [206]:
adjp_list_appear_more_deep

['very slow',
 'so good',
 'pleasantly surprised',
 'so great',
 'so friendly',
 'just ok',
 'pretty good',
 'pick up',
 'able to order',
 'any more',
 'friendly and informative',
 'a little pricey',
 'so hungry']

### for shallow extraction

In [196]:
adjp_list_appear_more_shallow = get_adjp_appear_more_than_n_times(random_200_reviews, indicative_adjp_sf_shallow, 2)

Number of adjp that appear more than n times: 1


In [197]:
adjp_list_appear_more_shallow

['pretty good']

In [201]:
adjp_list_appear_more_shallow = get_adjp_appear_more_than_n_times(random_1000_reviews, indicative_adjp_sf_shallow, 2)

Number of adjp that appear more than n times: 7


In [203]:
adjp_list_appear_more_shallow

['very slow',
 'pleasantly surprised',
 'so great',
 'so friendly',
 'just ok',
 'pretty good',
 'so hungry']

In [207]:
adjp_list_appear_more_shallow = get_adjp_appear_more_than_n_times(random_2000_reviews, indicative_adjp_sf_shallow, 2)

Number of adjp that appear more than n times: 8


In [208]:
adjp_list_appear_more_shallow

['very slow',
 'pleasantly surprised',
 'so great',
 'so friendly',
 'just ok',
 'pretty good',
 'a little pricey',
 'so hungry']

# Output to csv file for manually anaylsis

### Indicative adjp

In [130]:
list_to_csv(indicative_adjp_sf_deep, '../result/deep_extraction_indicative_adjp_result.csv')
list_to_csv(indicative_adjp_sf_shallow, '../result/shallow_extraction_indicative_adjp_result.csv')

### Sentence in biz

In [182]:
# split into sentences for manual review & output to csv
reviews_from_biz_split = split_into_sentence(all_reviews, '../result/biz_review.csv')

0
10
20
30
40
50
60
70
80
90


In [183]:
# Using DataFrame.insert() to add a column
reviews_from_biz_split.insert(0, "Indicative", [False for i in range(len(reviews_from_biz_split))], True)
reviews_from_biz_split.insert(2, "ADJP - deep extraction", ['-' for i in range(len(reviews_from_biz_split))], True)
reviews_from_biz_split.insert(3, "ADJP - shallow extraction", ['-' for i in range(len(reviews_from_biz_split))], True)
reviews_from_biz_split.rename(columns={0: "Sentences in biz"}, inplace=True)
reviews_from_biz_split.head()

,Indicative,Sentences in biz,ADJP - deep extraction,ADJP - shallow extraction
0,False,Disappointed in my Waffle.,-,-
1,False,After my third cup of coffee my waffle arrived.,-,-
2,False,It was obviously sitting under some sort of wa...,-,-
3,False,I waited so long for my waffle I didn't have t...,-,-
4,False,I ate less than half and didn't want to eat an...,-,-


In [188]:
deep_indication_str = '\t'.join(indicative_adjp_sf_deep)
for index, row in reviews_from_biz_split.iterrows():
#     print(index)
    for adjp in indicative_adjp_sf_deep:
        if adjp in row["Sentences in biz"].lower():
#             reviews_from_biz_split.loc[index, 'TOC']
            reviews_from_biz_split.loc[index, 'ADJP - deep extraction'] = adjp
            reviews_from_biz_split.loc[index, 'Indicative'] = True
    for adjp in indicative_adjp_sf_shallow:
        if adjp in row["Sentences in biz"].lower():
#             print(adjp)
            reviews_from_biz_split.loc[index, 'ADJP - shallow extraction'] = adjp
            reviews_from_biz_split.loc[index, 'Indicative'] = True




In [189]:
reviews_from_biz_split.head()

,Indicative,Sentences in biz,ADJP - deep extraction,ADJP - shallow extraction
0,True,Disappointed in my Waffle.,disappointed in my waffle,disappointed in my waffle
1,False,After my third cup of coffee my waffle arrived.,-,-
2,False,It was obviously sitting under some sort of wa...,-,-
3,True,I waited so long for my waffle I didn't have t...,so long for my waffle,so long for my waffle
4,True,I ate less than half and didn't want to eat an...,any more,-


In [190]:
reviews_from_biz_split.to_csv('../result/business_adjp_analysis.csv')